# 주의사항
1. windows 환경에서는 빌드된 ffmpeg 실행파일을 환경 변수 > Path 에 등록해야 한다.
2. gym 0.18.0 에는 video로 저장할 수 없는 버그가 있다. [링크](https://github.com/openai/gym/issues/1925)

[참조](https://wegonnamakeit.tistory.com/59)

In [1]:
import gym
from gym import wrappers
from IPython.display import Video

import collections
import random

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#Hyberparameters
learning_rate = 0.0005
gamma = 0.98
buffer_limit = 50000
batch_size = 32

In [2]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [3]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()

In [4]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [5]:
def is_record(episode_id):
    # print(episode_id, end=' ')
    if episode_id in is_record.st:
        # print('<< 녹화 ', end=' ')
        return True
    else:
        return False
is_record.st = set()

def main():
    n_episode = 10000  # 에피소드 수
    n_train_size = 2000  # 샘플  최소 이상일 때 학습
    max_score = 500
    env = gym.make('CartPole-v1')
    env = wrappers.Monitor(env, "./gym-results", force=True, video_callable=is_record)
    env._max_episode_steps = 10001
    
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    video_interval = 50
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    score_li = []
    is_end = -1

    for n_epi in range(n_episode):

        if is_end > 0:
            is_end -= 1
            if is_end == 0:
                break 
        
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()  # 현재 비디오 저장???
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            s_prime, r, done, info = env.step(a)  # 여기서 단계 진행하고 return되기 전에 프레임 저장??
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime

            score += r
            if done:
                break

        if memory.size()>n_train_size:
            train(q, q_target, memory, optimizer)

        if n_epi % print_interval == 0 and n_epi != 0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score_li.append(score/print_interval)
            if (is_end == -1 and score/print_interval >= max_score):
                is_record.st.add(n_epi + 1)
                print(n_epi + 1, "종료 점수: ", score/print_interval)
                is_end = 2  # 직후에 종료
            score = 0.0
        
        # if not (n_epi + 1) % video_interval:
        #     is_record.st.add(n_epi + 1)
    env.close()
    return env, n_episode

if __name__ == '__main__':
    env, n_episode = main()
    # print(is_record.st)


n_episode :20, score : 10.2, n_buffer : 205, eps : 7.9%
n_episode :40, score : 9.7, n_buffer : 398, eps : 7.8%
n_episode :60, score : 9.9, n_buffer : 597, eps : 7.7%
n_episode :80, score : 9.4, n_buffer : 786, eps : 7.6%
n_episode :100, score : 10.0, n_buffer : 986, eps : 7.5%
n_episode :120, score : 9.9, n_buffer : 1184, eps : 7.4%
n_episode :140, score : 9.8, n_buffer : 1380, eps : 7.3%
n_episode :160, score : 10.1, n_buffer : 1582, eps : 7.2%
n_episode :180, score : 9.7, n_buffer : 1775, eps : 7.1%
n_episode :200, score : 9.9, n_buffer : 1974, eps : 7.0%
n_episode :220, score : 11.9, n_buffer : 2212, eps : 6.9%
n_episode :240, score : 10.1, n_buffer : 2414, eps : 6.8%
n_episode :260, score : 9.4, n_buffer : 2602, eps : 6.7%
n_episode :280, score : 10.0, n_buffer : 2802, eps : 6.6%
n_episode :300, score : 12.2, n_buffer : 3047, eps : 6.5%
n_episode :320, score : 14.8, n_buffer : 3342, eps : 6.4%
n_episode :340, score : 14.8, n_buffer : 3639, eps : 6.3%
n_episode :360, score : 72.2, n

In [6]:
# def main():
#     n_episode = 5000  # 에피소드 수
#     n_train_size = 2000  # 샘플  최소 이상일 때 학습
#     max_score = 500
#     env = gym.make('CartPole-v1')
#     env = wrappers.Monitor(env, "./gym-results", force=True, video_callable=lambda episode_id: episode_id == n_episode - 1)

#     q = Qnet()
#     q_target = Qnet()
#     q_target.load_state_dict(q.state_dict())
#     memory = ReplayBuffer()

#     print_interval = 20
#     score = 0.0  
#     optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
#     score_li = []

#     for n_epi in range(n_episode):
#         epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
#         s = env.reset()  # 현재 비디오 저장
#         done = False

#         while not done:
#             a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
#             s_prime, r, done, info = env.step(a)  # 여기서 단계 진행하고 return되기 전에 프레임 저장
#             done_mask = 0.0 if done else 1.0
#             memory.put((s,a,r/100.0,s_prime, done_mask))
#             s = s_prime

#             score += r
#             if done:
#                 break
            
#         if memory.size()>n_train_size:
#             train(q, q_target, memory, optimizer)

#         if n_epi % print_interval == 0 and n_epi != 0:
#             q_target.load_state_dict(q.state_dict())
#             print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
#                                                             n_epi, score/print_interval, memory.size(), epsilon*100))
#             score_li.append(score/print_interval)
#             if (score/print_interval >= max_score):
#                 break
#             score = 0.0
#     env.close()
#     return env, n_episode

# if __name__ == '__main__':
#     env, n_episode = main()


In [7]:

# video_url = './gym-results/openaigym.video.{0}.video{1:0>6}.mp4'.format(env.file_infix, n_episode-1)
# Video(video_url)